In [18]:
import fasttext

model = fasttext.train_unsupervised('twitter-datasets/train_merged.txt', model='cbow')

In [19]:
model.save_model("models/fasttext_cbow_train.bin")


In [20]:
model = fasttext.load_model("models/fasttext_cbow_train.bin")

In [21]:
import numpy as np
import pandas as pd

# Load positive tweets
pos_tweets = []
with open('twitter-datasets/train_pos_full_u.txt', encoding = 'utf-8') as f:
    for line in f:
        pos_tweets.append(model[line])
        
# Load negative tweets
neg_tweets = []
with open('twitter-datasets/train_neg_full_u.txt', encoding = 'utf-8') as f:
    for line in f:
        neg_tweets.append(model[line])
        
pos_tweets = np.array(pos_tweets)
neg_tweets = np.array(neg_tweets)

In [22]:
all_tweets = np.concatenate((pos_tweets, neg_tweets))

In [23]:
y = np.concatenate((np.ones(len(pos_tweets)), np.zeros(len(neg_tweets))))

random_idxs = np.random.permutation(len(y))

all_tweets = all_tweets[random_idxs]

y = y[random_idxs]

N_train = int(0.8*len(y))

train, val = all_tweets[:N_train], all_tweets[N_train:]
y_train, y_val = y[:N_train], y[N_train:]

In [24]:
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0, tol=1e-9, loss = 'squared_hinge', dual = True, C = 0.03)
clf.fit(train, y_train)
train_acc = (clf.predict(train) == y_train).mean()
val_acc = (clf.predict(val) == y_val).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))

Training set accuracy: 72.84% / validation set: 72.79%
